In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [20]:
def update_section_name(section_name):
    if "data sheet" in section_name.lower():
        section_name = "Data sheet"
    elif "application note" in section_name.lower():
        section_name = "Application note"
    elif "technical note" in section_name.lower():
        section_name = "Technical note"
    elif "manual" in section_name.lower():
        section_name = "Manual"
    elif "article" in section_name.lower():
        section_name = "Article"
    elif "end-user documentation" in section_name.lower():
        section_name = "End-user documentation"
    else:
        section_name = section_name
    return section_name

In [21]:
different_section_names = []


In [22]:
df = pd.read_excel("./data/all_products_18_12_2.xlsx")


In [23]:
def find_downloads(url):
    response = requests.get(url)
    html = response.text

    # Parse the HTML
    soup = BeautifulSoup(html, 'html.parser')
    hrefs = {"Heading": "Downloads",
            "Sections": []}

    for subtitle in soup.find_all("h3", class_="Section__subtitle"):
        section_name = subtitle.text.strip()
        section_name = update_section_name(section_name)
        if section_name == 'Stay in touch for product information and software updates':
            break
        if section_name not in different_section_names:
            different_section_names.append(section_name)
        
        downloads_names = [span.text.strip() for span in subtitle.find_next_sibling("ul").find_all("span")]
        links = [a['href'] for a in subtitle.find_next_sibling("ul").find_all("a", href=True)]

        link_list = []
        for name, link in zip(downloads_names, links):
            if not link.endswith(".aspx") and not link.startswith("~") and not link.startswith("#"):
                link_dict = {
                    "Text": name,
                    "Href": link,
                    "Title": None,
                    "Target": None}
                link_list.append(link_dict)
        if link_list:
            hrefs["Sections"].append({"Heading": section_name, "Links": link_list})

        # hrefs = {"Heading": "Downloads", "Sections": [{"Heading": name, "Links": 'https://www.kongsberg.com' + link if link.startswith('/') else link for name, link in zip(downloads_name, links)}]

    

    for section in soup.find_all("section"):
        links = []
        downloads_names = []
        title_tag = soup.find('h2', class_='Section__title', text='Additional product information')
        download_tag = soup.find('h2', class_='Section__title', text='Downloads')
        if title_tag and not hrefs["Sections"]:
            section_name = title_tag.text.strip()
            if section_name not in different_section_names:
                different_section_names.append(section_name)            
            for link_tag in section.find_all("a", href=True):
                link_href = link_tag['href']
                links.append(link_href)
                if link_tag.find("span"):
                    downloads_names.append(link_tag.find("span").text.strip())
                else:
                    downloads_names.append(link_tag.text.strip())

        elif download_tag and not hrefs["Sections"]:
            section_name = download_tag.text.strip()
            if section_name not in different_section_names:
                different_section_names.append(section_name)
            downloads_container = download_tag.find_next_sibling("div")
            for link_tag in downloads_container.find_all("a", href=True):
                link_href = link_tag['href']
                links.append(link_href)
                if link_tag.find("span"):
                    downloads_names.append(link_tag.find("span").text.strip())
                else:
                    downloads_names.append(link_tag.text.strip())
            
        else:
            break
        link_list = []
        for name, link in zip(downloads_names, links):
            if not link.endswith(".aspx") and not link.startswith("~") and not link.startswith("#"):
                link_dict = {
                    "Text": name,
                    "Href": link,
                    "Title": None,
                    "Target": None}
                link_list.append(link_dict)
        if link_list:
            hrefs["Sections"].append({"Heading": section_name, "Links": link_list})
    if hrefs["Sections"]:
        return hrefs
    return None


In [24]:
df["downloads"] = df.apply(lambda row: find_downloads(row["url"]) if pd.notna(row["url"]) else {"Heading": "Downloads", 
                                                                                                "Sections": [{
                                                                                                    "Heading": "Data sheet", "Links": [{
                                                                                                                                    "Text": "Data sheet",
                                                                                                                                    "Href": row["Data sheets"],
                                                                                                                                    "Title": None,
                                                                                                                                    "Target": None}]}]}, axis=1)


C:\Users\HerborgHermansenTvei\AppData\Local\Temp\ipykernel_15248\3796367043.py:40: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  title_tag = soup.find('h2', class_='Section__title', text='Additional product information')
C:\Users\HerborgHermansenTvei\AppData\Local\Temp\ipykernel_15248\3796367043.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  download_tag = soup.find('h2', class_='Section__title', text='Downloads')


In [ ]:
def dict_to_json(row):
    return json.dumps(row)

# Apply the function to the 'downloads' column
df['downloads'] = df['downloads'].apply(dict_to_json)

In [ ]:
df.to_excel("./data/all_products_18_12_2.xlsx")